## 개 고양이 분류
- https://www.kaggle.com/sarvajna/dogs-vs-cats-keras-solution

In [3]:
# 필요한 라이브러리
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split

In [5]:
# Data dimensions and paths
img_width = 150
img_height = 150
TRAIN_DIR = '../dataset/dogs-vs-cats/train/'
TEST_DIR = '../dataset/dogs-vs-cats/test1/'
train_images_dogs_cats = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset
test_images_dogs_cats = [TEST_DIR+i for i in os.listdir(TEST_DIR)]

In [6]:
# Helper function to sort the image files based on the numeric value in each file name.
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

- Sort the traning set. 
- Use 1300 images each of cats and dogs instead of all 25000 to speed up the learning process.
- Sort the test set

In [7]:
train_images_dogs_cats.sort(key=natural_keys)
train_images_dogs_cats = train_images_dogs_cats[0:1300] + train_images_dogs_cats[12500:13800] 

test_images_dogs_cats.sort(key=natural_keys)

- Now the images have to be represented in numbers. 
- For this, using the openCV library read and resize the image.
- Generate labels for the supervised learning set.
- Below is the helper function to do so.

In [8]:
def prepare_data(list_of_images):
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels
    
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width,img_height), 
                            interpolation=cv2.INTER_CUBIC))
    
    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)
        #else:
            #print('neither cat nor dog name present in images')
            
    return x, y

- Generate X and Y using the helper function above
- Since K.image_data_format() is channel_last, input_shape to the first keras layer will be (img_width, img_height, 3). 
- '3' since it is a color image

In [9]:
X, Y = prepare_data(train_images_dogs_cats)
print(K.image_data_format())

channels_last


- Split the data set containing 2600 images into 2 parts, training set and validation set. 
- Later, you will see that accuracy and loss on the validation set will also be reported while fitting the model using training set.

In [10]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.2, random_state=1)

In [11]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 16

- We will be using the Sequential model from Keras to form the Neural Network. 
- Sequential Model is used to construct simple models with linear stack of layers.

In [13]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

- Prepare generators for training and validation sets

In [15]:
train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)

- Start training the model!

In [16]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

Epoch 1/30
130/130 [==============================] - 36s 276ms/step - loss: 0.0060 - acc: 0.9928 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/30
130/130 [==============================] - 35s 271ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/30
130/130 [==============================] - 35s 270ms/step - loss: 1.1935e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/30
130/130 [==============================] - 35s 273ms/step - loss: 1.1947e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/30
130/130 [==============================] - 35s 271ms/step - loss: 1.2061e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 6/30
130/130 [==============================] - 35s 272ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 7/30
130/130 [==============================] - 35s 272ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epo

In [17]:
model.save('model_keras.h5')

In [18]:
X_test, Y_test = prepare_data(test_images_dogs_cats) #Y_test in this case will be []

In [19]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [20]:
test_generator = val_datagen.flow(np.array(X_test), batch_size=batch_size)
prediction_probabilities = model.predict_generator(test_generator, verbose=1)

ValueError: `steps=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps` or use the `keras.utils.Sequence` class.